# Collecting description of offers

In [128]:
import pandas as pd
import requests
import numpy as np

In [129]:
file_name="df_garages_2022-12-22.gzip"

In [130]:
df=pd.read_parquet(file_name, columns=["hash_id"])
df.head()

,hash_id
0,2670483276
1,2495821644
2,156955724
3,2362082380
4,377640780


In [131]:
desc_indices=df.hash_id
desc_indices

0      2670483276
1      2495821644
2       156955724
3      2362082380
4       377640780
          ...    
514     818410572
515    3666632028
516    1207766108
517    2049567836
518     619855452
Name: hash_id, Length: 519, dtype: int64

In [132]:
desc_indices.drop_duplicates()
desc_indices.shape

(519,)

In [133]:
collector_description={}

for i in range(len(desc_indices)):
    base_url = 'https://www.sreality.cz/api/cs/v2/estates/{}'.format(desc_indices[i])

    r = requests.get(base_url)

    #print("starting sleep")
    #sleep(randint(0,2))

    if r.status_code==404:
        break
    
    elif r.status_code==200:
        r_dict=r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
        print(f"Description {i+1} out of {len(desc_indices)} was scraped. hash_id: {desc_indices[i]}")

Description 1 out of 519 was scraped. hash_id: 2670483276
Description 2 out of 519 was scraped. hash_id: 2495821644
Description 3 out of 519 was scraped. hash_id: 156955724
Description 4 out of 519 was scraped. hash_id: 2362082380
Description 5 out of 519 was scraped. hash_id: 377640780
Description 6 out of 519 was scraped. hash_id: 3565311820
Description 7 out of 519 was scraped. hash_id: 171242572
Description 8 out of 519 was scraped. hash_id: 571598668
Description 9 out of 519 was scraped. hash_id: 68518988
Description 10 out of 519 was scraped. hash_id: 1904604236
Description 11 out of 519 was scraped. hash_id: 2306889548
Description 12 out of 519 was scraped. hash_id: 288511820
Description 13 out of 519 was scraped. hash_id: 3232671308
Description 14 out of 519 was scraped. hash_id: 424527692
Description 15 out of 519 was scraped. hash_id: 3310348108
Description 16 out of 519 was scraped. hash_id: 3821401164
Description 17 out of 519 was scraped. hash_id: 1812329548
Description 18

In [ ]:
items_dict={"Zlevněno":"discounted", 
"Původní cena":"price_original", 
"ID zakázky":"id_order", 
"Aktualizace":"date_update", 
"Stavba":"building_type", 
"Stav objektu":"building_condition", 
"Umístění objektu":"building_location", 
"Užitná plocha":"area_net", 
"Plocha podlahová":"area_floor", 
"Celková cena":"price_total", 
"Poznámka k ceně":"price_note", 
"Doprava":"transport", 
"Plocha zastavěná":"area_build_up", 
"Datum nastěhování":"date_moving_in", 
"Rok kolaudace":"date_building_approval", 
"Vybavení":"equipped", 
"Elektřina":"electricity", 
"Komunikace":"traffic_communication", 
"Bezbariérový":"barrier_free", 
"ID":"id", 
#"Cena":"price", 
"Energetická náročnost budovy":"energy_efficient_rating", 
"Výtah":"lift", 
"Typ domu":"building_floor_type", 
"Podlaží":"building_floor", 
"Voda":"water", 
"Plyn":"gas", 
"Odpad":"waste", 
"Topení":"heating", 
"Telekomunikace":"telecommunication", 
"Rok rekonstrukce":"date_year_reconstruction", 
"Průkaz energetické náročnosti budovy":"energy_efficient_rating_card", 
"Stav":"availability", 
"Datum zahájení prodeje":"date_offer_start", 
"Ukazatel energetické náročnosti budovy":"energy_efficient_rating_index", 
"Datum prohlídky":"date_tour", 
"Datum prohlídky do":"date_tour_to", 
"Plocha zahrady":"area_garden"}

In [ ]:
description_individual = {}
r_dict_names_all=pd.Series(dtype="object")
r_dict_types_all=pd.DataFrame(columns=["name", "type"])

for page, r_dict in collector_description.items():
        
        info_relevant = pd.Series(dtype="object")
        info_relevant["description"]=r_dict["text"]["value"]

        r_dict_values=pd.DataFrame(r_dict["items"], columns =['type', 'name', 'value'])

        r_dict_names=r_dict_values["name"]
        r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])

        r_dict_types=r_dict_values[["type", "name"]]
        r_dict_types_all=pd.concat([r_dict_types_all, r_dict_types.loc[~r_dict_types["name"].isin(r_dict_types_all["name"]),:]])

        for name_raw in items_dict.keys():

                name_clean=items_dict[name_raw]

                if name_raw not in r_dict_names.values:
                        info_relevant[name_clean]=np.nan
                elif r_dict_values[r_dict_names==name_raw]["type"].all()=="set":
                        index_nr=int(r_dict_values.index[r_dict_values['name'] == name_raw].tolist()[0])
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw][index_nr][0]["value"].values[0]
                        del index_nr
                else:
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw].values[0]

        #info_relevant["price"]=r_dict_values["value"][r_dict_values["name"]=="Cena"]
        #info_relevant["price_note"]=r_dict_values["value"][r_dict_values["name"]=="Poznámka k ceně"]
        #info_relevant["price_per_sqm"]=r_dict_values["value"][r_dict_values["name"]=="Cena za m²"]
        #info_relevant["id_order"]=r_dict_values["value"][r_dict_values["name"]=="ID zakázky"]
        #info_relevant["date_update"]=r_dict_values["value"][r_dict_values["name"]=="Aktualizace"]
        #info_relevant["building_type"]=r_dict_values["value"][r_dict_values["name"]=="Stavba"]
        #info_relevant["availability"]=r_dict_values["value"][r_dict_values["name"]=="Stav"]
        #info_relevant["building_condition"]=r_dict_values["value"][r_dict_values["name"]=="Stav objektu"]
        #info_relevant["building_location"]=r_dict_values["value"][r_dict_values["name"]=="Umístění objektu"]
        #info_relevant["building_floor_type"]=r_dict_values["value"][r_dict_values["name"]=="Typ domu"]
        #info_relevant["building_floor"]=r_dict_values["value"][r_dict_values["name"]=="Podlaží"]
        #info_relevant["date_moving_in"]=r_dict_values["value"][r_dict_values["name"]=="Datum nastěhování"]
        #info_relevant["area_build_up"]=r_dict_values["value"][r_dict_values["name"]=="Plocha zastavěná"]
        #info_relevant["garage"]=r_dict_values["value"][r_dict_values["name"]=="Garáž"]
        #info_relevant["parking"]=r_dict_values["value"][r_dict_values["name"]=="Parkování"]    

        #if sum(r_dict_values['name'] == "Topení")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Topení"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Topení"][index_nr][0]["value"]
        #        del index_nr

        #if sum(r_dict_values['name'] == "Doprava")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Doprava"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Doprava"][index_nr][0]["value"]
        #        del index_nr

        #if sum(r_dict_values['name'] == "Telekomunikace")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Telekomunikace"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Telekomunikace"][index_nr][0]["value"]
        #        del index_nr

        #if sum(r_dict_values['name'] == "Plyn")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Plyn"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Plyn"][index_nr][0]["value"]
        #        del index_nr

        #info_relevant["energy_efficient_rating"]=r_dict_values["value"][r_dict_values["name"]=="Energetická náročnost budovy"]
        #info_relevant["lift"]=r_dict_values["value"][r_dict_values["name"]=="Výtah"]
        #info_relevant["equipped"]=r_dict_values["value"][r_dict_values["name"]=="Vybavení"]

        description_individual[r_dict["hash_id"]] = info_relevant

/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/3661897441.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/3661897441.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858/3661897441.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_46858

In [ ]:
r_dict_types_all

,name,type
0,Zlevněno,price_czk
1,Původní cena,price_czk_old
2,ID zakázky,string
3,Aktualizace,edited
4,Stavba,string
5,Stav objektu,string
6,Umístění objektu,string
7,Užitná plocha,area
8,Plocha podlahová,area
0,Celková cena,price_czk


In [ ]:
r_dict_types_all
boolean_formats=["boolean"]
float_formats=["integer", "price_czk", "price_czk_old", "area", ""]

0                         price_czk
1                     price_czk_old
2                            string
3                            edited
4                            string
5                            string
6                            string
7                              area
8                              area
7                               set
9                           integer
10                          boolean
7                              date
0                        price_info
9          energy_efficiency_rating
10    energy_performance_attachment
10               energy_performance
dtype: object

In [ ]:
r_dict_values["value"][r_dict_values["name"]==name_raw]

Series([], Name: value, dtype: object)

In [ ]:
print("Add these values to your dictionary:")
print(r_dict_names_all[~r_dict_names_all.isin(items_dict.keys())])

Add these values to your dictionary:
0    Cena
dtype: object


In [ ]:
df_desc = pd.concat(description_individual).unstack()
df_desc.head()

,description,discounted,price_original,id_order,date_update,building_type,building_condition,building_location,area_net,area_floor,...,heating,telecommunication,date_year_reconstruction,energy_efficient_rating_card,availability,date_offer_start,energy_efficient_rating_index,date_tour,date_tour_to,area_garden
2670483276,Prodávám garážové stání v osobním vlastnictví ...,1 225 000,1 334 000,GR145386,Dnes,Cihlová,Velmi dobrý,Klidná část obce,21,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2495821644,Nabízím k prodeji zděnou garáž o velikosti 16 ...,NaN,NaN,14425911,Dnes,Cihlová,Dobrý,NaN,21,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156955724,Nabízíme garážové stání k pronájmu ve druhém p...,NaN,NaN,0384,Dnes,Cihlová,Velmi dobrý,Centrum obce,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2362082380,Nabízíme k prodeji parkovací stání o rozměrech...,NaN,NaN,PS.22,Dnes,Smíšená,Novostavba,Klidná část obce,17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377640780,Společnost Home Real Estate nabízí na prodej v...,NaN,NaN,36008,Dnes,Cihlová,Velmi dobrý,NaN,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_desc["equipped"]=df_desc["equipped"].map({True: "ano", False: "ne", "Částečně":"částečně"}) 

### Saving data

In [ ]:
df_desc.to_parquet((file_name[:-5]+"_desc.gzip")) 

In [ ]:
break

# New version of scraping - proxies and multiple requests at the same time

In [ ]:
from fp.fp import FreeProxy
import gevent
from gevent import monkey, pool
#https://stackoverflow.com/questions/15322701/gevent-pool-with-nested-web-requests
#https://pypi.org/project/free-proxy/

In [ ]:
proxy = FreeProxy(country_id=['DE', "CZ", "BR"]).get()

In [ ]:
urls=np.add("https://www.sreality.cz/api/cs/v2/estates/", desc_indices)

In [ ]:
monkey.patch_all()

jobs = []
links = []
p = pool.Pool(10)

add

urls = [
    'http://www.google.com', 
    # ... another 100 urls
]
    
def get_jsons(url):
    r = requests.get(url)
    if r.status_code == 200:
        r_dict = r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
for url in urls:
    jobs.append(p.spawn(get_jsons, url))
gevent.joinall(jobs)